<a href="https://colab.research.google.com/github/svnnynior/introduction-to-langchain-workshop/blob/main/introduction-to-langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Langchain

LangChain is a framework for developing applications powered by language models

- GitHub: https://github.com/langchain-ai/langchain
- Docs: https://python.langchain.com/docs/get_started

## Outlines
1. Main components -- Model, Prompt Template, Output Parser
2. Chains
3. Memory
4. Retriever (RAG)
5. Evaluation


## 0. Installation

In [1]:
!pip install langchain==0.0.349

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.6/808.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.9 MB/s eta 0:00:00


## 1. Main Components

### 1.1. Model


https://python.langchain.com/docs/modules/model_io/

#### Model - Hugging Face (google/flan-t5-xxl)

In [2]:
!pip install huggingface_hub==0.19.4

In [4]:
import os
from langchain.llms import HuggingFaceHub
from google.colab import userdata

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options


llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [5]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))

Socks for Less


#### Model - Local (LlamaCPP)

In [4]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [5]:
!wget https://gpt4all.io/models/gguf/mistral-7b-instruct-v0.1.Q4_0.gguf

--2023-12-13 16:12:17--  https://gpt4all.io/models/gguf/mistral-7b-instruct-v0.1.Q4_0.gguf
Resolving gpt4all.io (gpt4all.io)... 172.67.71.169, 104.26.1.159, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|172.67.71.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4108916384 (3.8G)
Saving to: ‘mistral-7b-instruct-v0.1.Q4_0.gguf.1’

mistral-7b-instruct 100%[===================>]   3.83G  68.6MB/s    in 81s     

2023-12-13 16:13:41 (48.4 MB/s) - ‘mistral-7b-instruct-v0.1.Q4_0.gguf.1’ saved [4108916384/4108916384]



In [2]:
from langchain.llms import LlamaCpp

llm = LlamaCpp(
    model_path="/content/mistral-7b-instruct-v0.1.Q4_0.gguf",
    n_gpu_layers=200,
    n_ctx=32000,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [3]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



A few ideas:

1. Socktopia (play on the word "utopia")
2. ColorCrew (like a crew of colors)
3. RainbowRhythms (rainbows are associated with vibrant colors)
4. SockScents (socks that smell good too!)
5. ColorfulConfections (like candy)
6. BrightBites (like bites of color)
7. SockSaga (a story about the company's journey)
8. SockStory (storytelling through socks)
9. SockSparkle (sparkly socks)
10. SockSnuggle (soft, cozy socks)

These names are just suggestions and can be adapted or combined to create a unique name that reflects the company's values and vision.


#### Model - OpenAI

In [2]:
!pip install openai==1.3.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from google.colab import userdata

# Get it from https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

##### LLM Model

LLMs in LangChain refer to pure text completion models. The APIs they wrap take a string prompt as input and output a string completion.

In [4]:
from langchain.llms import OpenAI
from google.colab import userdata

text = "What would be a good company name for a company that makes colorful socks?"

llm = OpenAI(temperature=0)
print(f"Using LLM model: {llm.model_name}")

Using LLM model: text-davinci-003


In [5]:
text = "What would be a good company name for a company that makes colorful socks?"

print(llm(text))



Rainbow Socks Co.


##### Chat Model

Chat models are often backed by LLMs but tuned specifically for having conversations. And, crucially, their provider APIs use a different interface than pure text completion models. Instead of a single string, they take a list of chat messages as input. Usually these messages are labeled with the speaker (usually one of "System", "AI", and "Human"). And they return an AI chat message as output

In [6]:
from langchain.schema import HumanMessage

chat = ChatOpenAI()
print(f"Using chat model: {chat.model_name}")

Using chat model: gpt-3.5-turbo


In [7]:
messages = [HumanMessage(content="What would be a good company name for a company that makes colorful socks?")]

chat(messages)

AIMessage(content='SockSplash')

#### Bonus: Using LLM as a question-answering model

In [8]:
text = """Question: What would be a good company name for a company that makes colorful socks?

Let's think step by step.

Answer: """

In [9]:
print(llm(text))


1. Brainstorm words related to socks: cozy, comfy, colorful, vibrant, hues, shades, etc.
2. Brainstorm words related to color: rainbow, spectrum, kaleidoscope, prism, etc.
3. Combine the two ideas: Rainbow Cozies, Vibrant Hues, Kaleidoscope Socks, Prism Comfort, etc.


### 1.2. Prompt Templates

Prompt templates are pre-defined recipes for generating prompts for language models.

https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/

In [10]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Let's think step by step, and then summarize the final answer in this format:

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [11]:
prompt_text = prompt.format(question="What is a good name for a company that makes video games")
print(prompt_text)

Question: What is a good name for a company that makes video games

Let's think step by step, and then summarize the final answer in this format:

Answer: 


In [12]:
print(llm(prompt_text))


Step 1: Brainstorm words related to video games. Examples: Play, Fun, Adventure, Challenge, Create, Digital, Entertainment.

Step 2: Combine words to create a unique name. Examples: Playful Entertainment, Digital Challenge, Create Adventure.

Step 3: Choose the best name.

Answer: Playful Entertainment


### 1.3. Output Parser

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

In [13]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [14]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [15]:
prompt_template = PromptTemplate(template=review_template, input_variables=["text"],)

In [16]:
answer = llm(prompt_template.format(text=customer_review))
print(answer)



{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["slightly more expensive than the other leaf blowers out there", "worth it for the extra features"]
}


In [17]:
## Throw an error because answer is just a string - not dict
answer['gift']

TypeError: ignored

#### Using output parsers

In [18]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [19]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [20]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [21]:
review_template_with_instructions = """\
For the following text, extract the following information:

{format_instructions}

text: {text}
"""

In [22]:
prompt_template_with_output = PromptTemplate(template=review_template_with_instructions, input_variables=["text"], partial_variables={"format_instructions": format_instructions})

In [23]:
prompt_and_model = prompt_template_with_output | llm
output = prompt_and_model.invoke({"text": customer_review})

In [24]:
# Note: Sometimes, if the model cannot extract the information, this can throw an error
# -- which is good because we want to know early that the model cannot achieve the tasks we want it to

result = output_parser.invoke(output)
print(result)

{'gift': 'True', 'delivery_days': '2', 'price_value': "['slightly more expensive than the other leaf blowers out there', 'worth it for the extra features']"}


In [25]:
result["gift"]

'True'

### Bonus: Chaining Stuffs

Because all of the objects implements the `Runnable` interface. It can be chained together.

More info: https://python.langchain.com/docs/expression_language/why

In [26]:
output_chain = prompt_template_with_output | llm | output_parser

In [27]:
output_chain.invoke({"text": customer_review})

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "['slightly more expensive than the other leaf blowers out there', 'worth it for the extra features']"}

## 2. Chains

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

LangChain provides two high-level frameworks for “chaining” components. The legacy approach is to use the Chain interface. The updated approach is to use the LangChain Expression Language (LCEL).

https://python.langchain.com/docs/modules/chains/

### Old way

The legacy interface for “chained” applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

In [28]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt_template_with_output, llm=llm, output_parser=output_parser)

llm_chain.run(text=customer_review)

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "['slightly more expensive than the other leaf blowers out there', 'worth it for the extra features']"}

### New way

LCEL provides an intuitive and readable syntax for composition.

In [29]:
from langchain_core.runnables import RunnablePassthrough

chain = (
 { "text": RunnablePassthrough() }
 | prompt_template_with_output
 | llm
 | output_parser
)
chain.invoke(customer_review)

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "['slightly more expensive than the other leaf blowers out there', 'worth it for the extra features']"}

## 3. Memory

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation

https://python.langchain.com/docs/modules/memory/

### Manipulating the memory

In [30]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [31]:
print(memory.load_memory_variables({})['history'])

Human: hi!
AI: what's up?


In [32]:
memory.save_context({"input": "how yo doin'"}, {"output": "fine. thank you!"})

In [33]:
print(memory.load_memory_variables({})['history'])

Human: hi!
AI: what's up?
Human: how yo doin'
AI: fine. thank you!


In [34]:
template = """You are a nice chatbot having a conversation with a human.

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)

#### Without memory

In [35]:
from langchain import LLMChain

no_memory_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
)

In [36]:
no_memory_chain({"question": "Hello, My name is Junior"})

{'question': 'Hello, My name is Junior',
 'text': ' Hi Junior, nice to meet you! How can I help you today?'}

In [37]:
no_memory_chain({"question": "I have just introduced myself. What is my name?"})

{'question': 'I have just introduced myself. What is my name?',
 'text': ' Nice to meet you! What is your name?'}

#### With memory - LLMChain

In [38]:
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
with_memory_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory ## here - we are giving it a memory
)

In [39]:
with_memory_chain({"question": "ay yo!"})

{'question': 'ay yo!',
 'chat_history': '',
 'text': ' Hi there! How can I help you?'}

In [40]:
memory.save_context({"input": "how yo doin' My name is Junior. Nice to meet you."}, {"output": "Nice to meet you, Junior!. I am fine. Thank you!"})

In [41]:
# Some model, like Hugging Face's, might not always work

with_memory_chain({"question": "what is my name again?"})

{'question': 'what is my name again?',
 'chat_history': "Human: ay yo!\nAI:  Hi there! How can I help you?\nHuman: how yo doin' My name is Junior. Nice to meet you.\nAI: Nice to meet you, Junior!. I am fine. Thank you!",
 'text': ' Your name is Junior.'}

#### With memory - ConversationChain

In [42]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    verbose=False,
)

In [43]:
print(conversation.predict(input="how yo doin' My name is Junior. Nice to meet you."))

 Hi Junior, nice to meet you too! I'm doing great, thank you for asking. How about you?


In [44]:
print(conversation.predict(input="what is my name again?"))

 Your name is Junior. Is there anything else I can help you with?


### Bonus: Different types of Memory

More info: https://python.langchain.com/docs/modules/memory/types/

In [45]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationSummaryMemory(llm=llm) # Note that the ConversationSummaryMemory will need an LLM as an input to do the summarization
)

In [46]:
# Notice the prompt that ConversationChain format for us

conversation.predict(input="how yo doin' My name is Junior. Nice to meet you.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: how yo doin' My name is Junior. Nice to meet you.
AI:

> Finished chain.


" Hi Junior, nice to meet you too! I'm doing great, thank you for asking. How about you?"

In [50]:
conversation.predict(input="I like the color red. My favorite subject is Math.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Junior and the AI responds, introducing itself and asking how Junior is doing. Junior expresses that they like the color red and their favorite subject is Math, to which the AI responds by asking why Junior likes the color red and what they like about Math. The AI then asks Junior to tell them what their favorite subject is, and when Junior asks if the AI can guess, the AI apologizes and asks Junior to tell them what their favorite subject is.
Human: I like the color red. My favorite subject is Math.
AI:

> Finished chain.


" Hi Junior, I'm AI. Nice to meet you. How are you doing?\n\nI'm curious, why do you like the color red? What do you like about Math? Could you tell me what your favorite subject is? If you'd like, I can try to guess, but I apologize if I don't get it right."

In [51]:
conversation.predict(input="Can you guess my favorite subject?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Junior and the AI responds, introducing itself and asking how Junior is doing. Junior expresses that they like the color red and their favorite subject is Math, to which the AI responds by asking why Junior likes the color red, what they like about Math, and for Junior to tell them what their favorite subject is. The AI then offers to try and guess Junior's favorite subject, apologizing if it doesn't get it right.
Human: Can you guess my favorite subject?
AI:

> Finished chain.


" Sure, I'd be happy to try! What do you like about Math? What makes it your favorite subject?"

## 4. Retriever (RAG)

Many LLM applications require user-specific data that is not part of the model's training set. The primary way of accomplishing this is through Retrieval Augmented Generation (RAG)

More info: https://python.langchain.com/docs/modules/data_connection/

***Note: LlamaCPP model users should skip this, as it might crash the notebook due to limited resource***

In [52]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [53]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [54]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

#### Hugging Face Embeddings

In [51]:
!pip install sentence_transformers==2.2.2 faiss-cpu==1.7.4

In [52]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [53]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [54]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


#### LlamaCPP Embeddings

***Note: LlamaCPP model users should skip this, as it might crash the notebook due to limited resource***

In [43]:
!pip install chromadb==0.4.19 tiktoken==0.5.2

In [44]:
from langchain.embeddings import LlamaCppEmbeddings

embeddings = LlamaCppEmbeddings(model_path="/content/mistral-7b-instruct-v0.1.Q4_0.gguf")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

#### OpenAI Embeddings

In [55]:
!pip install sentence_transformers==2.2.2 faiss-cpu==1.7.4 tiktoken==0.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=bf0d3d463b4751dbcc9211d74f87da1fb7846e93fbbfaa7fd30f3650fb4a8fb0
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [56]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [57]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [58]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


#### Using Retriever

LangChain supports many different retrieval algorithms and is one of the places where it adds the most value

In [59]:
retriever = db.as_retriever()

In [60]:
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}

Let's think step by step.

Answer:
"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm

)

In [61]:
answer = chain.invoke("Who is Ketanji Brown Jackson?")
print(answer)

Ketanji Brown Jackson is a Circuit Court of Appeals Judge who was nominated by the President 4 days ago to serve on the United States Supreme Court. She is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. She has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.


In [62]:
import langchain

langchain.debug = True
chain.invoke("What does the speech say about Russia?")
langchain.debug = False

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What does the speech say about Russia?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What does the speech say about Russia?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableSequence > 5:chain:format_docs] Entering Chain run with input:
[inputs]
[chain

## 5. Evaluation

Building applications with language models involves many moving parts. One of the most critical components is ensuring that the outcomes produced by your models are reliable and useful across a broad array of inputs, and that they work well with your application's other software components.

More info: https://python.langchain.com/docs/guides/evaluation/


### Generating test datasets & evaluate its accuracy

In [63]:
!pip install langchain[docarray]==0.0.349 openai==1.3.9 huggingface_hub==0.19.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287867 sha256=3e1ffec675117b1ae753018ca8530405de3fa566def108a950a7d0bba263aff5
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [64]:
import requests

url = "https://raw.githubusercontent.com/Ryota-Kawamura/LangChain-for-LLM-Application-Development/main/OutdoorClothingCatalog_1000.csv"
res = requests.get(url)
with open("OutdoorClothingCatalog_1000.csv", "w") as f:
  f.write(res.text)

In [66]:
from langchain.llms import OpenAI
from langchain.llms import HuggingFaceHub
from langchain.chat_models import ChatOpenAI


os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

hugging_face_repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options


openai_llm = OpenAI(temperature=0)
hf_llm = HuggingFaceHub(
    repo_id=hugging_face_repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)
chat_model = ChatOpenAI()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [67]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA

file = "./OutdoorClothingCatalog_1000.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [70]:
print(data[10].page_content)

: 10
name: Cozy Comfort Pullover Set, Stripe
description: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.

Size & Fit
- Pants are Favorite Fit: Sits lower on the waist.
- Relaxed Fit: Our most generous fit sits farthest from the body.

Fabric & Care
- In the softest blend of 63% polyester, 35% rayon and 2% spandex.

Additional Features
- Relaxed fit top with raglan sleeves and rounded hem.
- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.

Imported.


In [71]:
test_data_examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
]

# But how can we automate the generation of these questions & answers ? (:thinking:)

In [72]:
from langchain.evaluation.qa import QAGenerateChain

example_gen_chain = QAGenerateChain.from_llm(chat_model)

In [73]:
generated_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [74]:
generated_examples[0]

{'qa_pairs': {'query': "What is the description of the Women's Campside Oxfords?",
  'answer': "The description of the Women's Campside Oxfords is that they are an ultracomfortable lace-to-toe Oxford made of super-soft canvas. They have thick cushioning and quality construction, providing a broken-in feel from the first time they are worn."}}

In [75]:
print(data[0].page_content)

: 0
name: Women's Campside Oxfords
description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. 

Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. 

Specs: Approx. weight: 1 lb.1 oz. per pair. 

Construction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. 

Questions? Please contact us for any inquiries.


In [77]:
# Define the model under test

qa = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
)

In [78]:
qa.run(test_data_examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'yes'

In [79]:
test_data = []
for example in generated_examples:
  test_data.append(example['qa_pairs'])

test_data

[{'query': "What is the description of the Women's Campside Oxfords?",
  'answer': "The description of the Women's Campside Oxfords is that they are an ultracomfortable lace-to-toe Oxford made of super-soft canvas. They have thick cushioning and quality construction, providing a broken-in feel from the first time they are worn."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size of the Recycled Waterhog Dog Mat, Chevron Weave has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
  'answer': "The features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bo

In [80]:
predictions = qa.apply(test_data)
predictions



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


[{'query': "What is the description of the Women's Campside Oxfords?",
  'answer': "The description of the Women's Campside Oxfords is that they are an ultracomfortable lace-to-toe Oxford made of super-soft canvas. They have thick cushioning and quality construction, providing a broken-in feel from the first time they are worn.",
  'result': 'This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on'},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size of the Recycled Waterhog Dog Mat, Chevron Weave has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".',
  'result': '18" x 28" and 22.5" x 34.5".'},
 {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece?",
  'answer': "The features of the Infant and Toddler Gi

`answer` is the actual answer expected from the question

`result` is what model under test predicts

Notice that the `answer` and the `result` is not an exact 1-to-1 match, BUT the content could be saying the same thing.

This is why we need **ANOTHER** LLM model to help evaluate whether the answer and the predicted result is saying the same thing.

In [81]:
from langchain.evaluation.qa import QAEvalChain

eval_chain = QAEvalChain.from_llm(openai_llm)

In [82]:
graded_outputs = eval_chain.evaluate(test_data, predictions)

In [83]:
for i, eg in enumerate(generated_examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print('Verdict: ' + graded_outputs[i]['results'])
    print()

Example 0:
Question: What is the description of the Women's Campside Oxfords?
Real Answer: The description of the Women's Campside Oxfords is that they are an ultracomfortable lace-to-toe Oxford made of super-soft canvas. They have thick cushioning and quality construction, providing a broken-in feel from the first time they are worn.
Predicted Answer: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on
Verdict:  CORRECT

Example 1:
Question: What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?
Real Answer: The small size of the Recycled Waterhog Dog Mat, Chevron Weave has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".
Predicted Answer: 18" x 28" and 22.5" x 34.5".
Verdict:  CORRECT

Example 2:
Question: What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Pi

### Bonus: Evaluator

In [ ]:
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

evaluator = load_evaluator(EvaluatorType.CRITERIA, criteria="conciseness")

eval_result = evaluator.evaluate_strings(
    prediction="What's 2+2? That's an elementary question. The answer you're looking for is that two and two is four.",
    input="What's 2+2?",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): N
Evaluation score: 0
Evaluation reasoning: The criterion is conciseness, which means the submission should be brief and to the point. 

Looking at the submission, the answer to the question "What's 2+2?" is given as "The answer you're looking for is that two and two is four." However, before providing the answer, the respondent adds an unnecessary comment: "That's an elementary question." This comment does not contribute to answering the question and thus makes the response less concise.

Therefore, the submission does not meet the criterion of conciseness.

N


In [ ]:
eval_result = evaluator.evaluate_strings(
    prediction="four.",
    input="What's 2+2?",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): Y
Evaluation score: 1
Evaluation reasoning: The criterion is conciseness, which means the submission should be brief and to the point. 

Looking at the submission, the answer to the question "What's 2+2?" is given as "four." This is a direct and succinct response to the question. 

The submission does not include any unnecessary information or details, making it concise. 

Therefore, the submission meets the criterion of conciseness. 

Y


#### Different type of evaluator -- Labeled Criteria

In [ ]:
evaluator = load_evaluator("labeled_criteria", criteria="correctness")

# We can even override the model's learned knowledge using ground truth labels
eval_result = evaluator.evaluate_strings(
    input="What is the capital of the US?",
    prediction="Bangkok",
    reference="The capital of the US is Washington D.C.",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): N
Evaluation score: 0
Evaluation reasoning: The criterion for this task is the correctness of the submitted answer. The input asks for the capital of the US. The submitted answer is "Bangkok", which is incorrect as the capital of the US is Washington D.C., as stated in the reference. Therefore, the submission does not meet the criterion of correctness.

N


In [ ]:
evaluator = load_evaluator("labeled_criteria", criteria="correctness")

# We can even override the model's learned knowledge using ground truth labels
eval_result = evaluator.evaluate_strings(
    input="What is the capital of the US?",
    prediction="Washington D.C.",
    reference="The capital of the US is Washington D.C.",
)

print(f'Evaluation value (Y/N): {eval_result["value"]}')
print(f'Evaluation score: {eval_result["score"]}')
print(f'Evaluation reasoning: {eval_result["reasoning"]}')

Evaluation value (Y/N): Y
Evaluation score: 1
Evaluation reasoning: The criterion for this task is the correctness of the submitted answer. This involves checking if the submission is accurate, factual, and directly answers the given input.

The input asks for the capital of the US. The submitted answer is Washington D.C.

Comparing this with the reference answer, which is also Washington D.C., it is clear that the submitted answer is correct. It is factual and accurate, as Washington D.C. is indeed the capital of the US.

Therefore, the submission meets the criterion.

Y
